In [32]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from datetime import date
from tqdm import tqdm_notebook
from tqdm import tqdm
tqdm_notebook().pandas()

import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import DBSCAN

A Jupyter Widget

In [33]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [13]:
train.rename(columns={'pos_adress_lat':'pos_address_lat', 'pos_adress_lon':'pos_address_lon'}, inplace=True)

In [14]:
data = pd.concat([train,test], axis=0)
data.shape

(2490204, 21)

In [15]:
data.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,date_int,dow,hol,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
0,2.884034,NaN,NaN,NaN,st petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,195.0,5.0,1,59.851,30.232,5261,NaN,59.844072,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177
1,2.775633,NaN,NaN,NaN,st petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,299.0,4.0,0,59.851,30.232,5261,NaN,59.844072,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177
2,3.708368,NaN,NaN,NaN,st petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,275.0,1.0,0,59.851,30.232,5992,"pr.marshala zhukova,31st petersburg190000 7...",59.858198,30.229024,df06c1fcd3718a514535ae822785f716,2017-10-03,59.847,30.177
3,2.787498,NaN,NaN,NaN,st petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,251.0,5.0,1,59.851,30.232,5261,NaN,59.844072,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177
4,2.892510,NaN,NaN,NaN,st petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,186.0,3.0,0,59.851,30.232,5261,NaN,59.844072,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177


In [10]:
data['lat'] = data.progress_apply(lambda x: max(x['atm_address_lat'], x['pos_address_lat']), axis=1)
data['lon'] = data.progress_apply(lambda x: max(x['atm_address_lon'], x['pos_address_lon']), axis=1)

A Jupyter Widget

A Jupyter Widget

In [2]:
df = pd.read_csv('popularity.csv')

In [4]:
mcc_list2 = ['products', 'meal', 'bank', 'apteka', 'univermag','clothes']

r002 = []
r002_pr = []
r002_meal = []
r002_bank = []
r002_apt = []
r002_univ = []
r002_clo = []

for line in tqdm_notebook(np.array(df)):
    lat = line[3]
    lon = line[4]
    
    df['y'] = np.sqrt((df.lat - lat)**2 + (df.lon - lon)**2)
    r002.append(df[df.y < 0.01].shape[0])
    r002_pr.append(df[(df.y < 0.01) & ((df.mcc == 5411) | (df.mcc == 5499)) ].shape[0])
    r002_meal.append(df[(df.y < 0.01) & ((df.mcc == 5821) | (df.mcc == 5814)) ].shape[0])
    r002_bank.append(df[(df.y < 0.01) & ((df.mcc == 6011) | (df.mcc == 6010)) ].shape[0])
    r002_apt.append(df[(df.y < 0.01) & (df.mcc == 5912)].shape[0])
    r002_univ.append(df[(df.y < 0.01) & ((df.mcc == 7832) | (df.mcc == 7932) | (df.mcc == 7933)) ].shape[0])
    r002_clo.append(df[(df.y < 0.01) & ((df.mcc == 5699) | (df.mcc == 5651) | (df.mcc == 5661) | (df.mcc == 5641)) ].shape[0])
    
    
df['r002'] = r002
df['r002_pr'] = r002_pr
df['r002_meal'] = r002_meal
df['r002_bank'] = r002_bank
df['r002_apt'] = r002_apt
df['r002_univ'] = r002_univ
df['r002_clo'] = r002_clo

A Jupyter Widget

In [34]:
import pickle
    
with open('train_coords.pcl', 'rb') as f:
     train_coord = pickle.load(f, encoding="latin-1")
with open('test_coords.pcl', 'rb') as f:
     test_coord = pickle.load(f, encoding="latin-1")

In [35]:
train_coord.string = train_coord.string.apply(lambda x: x.lower())
train_coord.string = train_coord.string.apply(lambda x: x.replace(',',' '))

test_coord.string = test_coord.string.apply(lambda x: x.lower())
test_coord.string = test_coord.string.apply(lambda x: x.replace(',',' '))

In [302]:
n = np.random.choice(range(train_coord.shape[0]), size=200, replace=False)

train_coord.iloc[n]

,action_lat,action_lon,string,tc,air,bc,vokzal,oao,health,metro,child,school,giper,city
24230,55.555149,37.712407,5 улица гаевского 6-й микрорайон городское ...,0,0,0,0,0,0,0,0,0,0,1
80276,59.851000,30.307000,marketplace 3а площадь конституции предпорт...,0,0,0,0,0,0,0,0,0,0,0
1286,59.914186,30.355056,продукты 42 тамбовская улица волково округ...,0,0,0,0,0,0,0,0,0,0,0
79512,59.867000,30.348000,бассейная улица округ гагаринское московский...,0,0,0,0,0,0,0,0,0,0,0
111674,59.895000,30.279000,сбербанк 18 проспект стачек нарвский округ ...,0,0,0,0,0,0,0,0,0,0,0
112250,55.746000,37.501000,6 сеславинская улица фили район филёвский п...,0,0,0,0,0,0,0,0,0,0,0
38659,55.778994,38.650839,1 большая покровская улица париж павловский...,0,0,0,0,0,0,0,0,0,0,0
55143,54.442709,45.829887,заводская улица комсомольское городское посел...,0,0,0,0,0,0,0,0,0,0,1
3223,55.960440,37.335968,кирилловка ленинградское шоссе кирилловка п...,0,0,0,0,0,0,0,0,0,0,0
74043,55.907000,37.541000,транскапиталбанк 161а дмитровское шоссе рай...,0,0,0,0,0,0,0,0,0,0,0


In [318]:
a = train_coord.iloc[['км ' in i for i in train_coord.string]]
print(a.shape[0])
a

312


,action_lat,action_lon,string,tc,air,bc,vokzal,oao,health,metro,child,school,giper,city
215,44.949810,38.927252,2 краснодар-новороссийск трасса 135 км энемс...,0,0,0,0,0,0,0,0,0,0,1
559,57.102107,65.714250,3 км старого тобольского тракта копытова лен...,0,0,0,0,0,0,0,0,0,0,0
1197,55.142247,36.659359,азс 99км копилка м-3 олимпийская деревня с...,0,0,0,0,0,0,0,0,0,0,1
1345,55.428782,37.277309,45 км калужское шоссе поселение краснопахорс...,0,0,0,0,0,0,0,0,0,0,1
1362,55.635269,37.437777,д вл4 кд 22 км киевского шоссе поселение мо...,0,0,0,0,0,0,0,0,0,0,1
2306,55.548062,37.118822,м-3 42 км киевское шоссе поселение первомайс...,0,0,0,0,0,0,0,0,0,0,1
2341,55.773872,37.584985,км 37 с1 2-я брестская улица тверской район...,0,0,0,0,0,0,0,0,0,0,0
2390,55.428953,37.277578,45 км калужское шоссе поселение краснопахорс...,0,0,0,0,0,0,0,0,0,0,1
2395,57.215702,59.301290,подъезд к р.п. староуткинск от км 65+100 а/д «...,0,0,0,0,0,0,0,0,0,0,0
2415,55.333267,37.528549,50 км симферопольское шоссе сельское поселен...,0,0,0,0,0,0,0,0,0,0,1


In [36]:
def tc(s):
    if(('тц ' in s) | ('торговый ' in s) | ('трц ' in s) | ('развлекательный' in s) | ('тк ' in s) | ('трк ' in s) | ('цум' in s)): return 1
    return 0

def air(s):
    if(('аэропорт' in s)): return 1
    return 0    
       
def bc(s):
    if(('бц ' in s) | ('бизнес' in s)): return 1
    return 0    

def vokzal(s):
    if(('вокзал ' in s)): return 1
    return 0   

def oao(s):
    if(('оао' in s) | ('ооо' in s)): return 1
    return 0 

def health(s):
    if(('поликлиник' in s) | ('больница' in s)): return 1
    return 0 

def metro(s):
    if(('metro ' in s) | ('леруа' in s) | ('икеа' in s) | ('castorama' in s)): return 1
    return 0 

def child(s):
    if(('детский сад' in s)): return 1
    return 0

def school(s):
    if(('школа' in s)): return 1
    return 0

def giper(s):
    if(('окей' in s) |('кей ' in s) |('магнит ' in s) |('дикси' in s) |('пятерочка' in s) |( 'ашан' in s) |('лента' in s) |('монетка' in s)): return 1
    return 0

def city(s):
    if(('поселок' in s) |  ('поселение' in s)): return 1
    return 0

def road(s):
    if(('км ' in s)): return 1
    return 0

In [37]:
train_coord['tc'] = train_coord.string.apply(lambda x: tc(x))
train_coord['air'] = train_coord.string.apply(lambda x: air(x))
train_coord['bc'] = train_coord.string.apply(lambda x: bc(x))
train_coord['vokzal'] = train_coord.string.apply(lambda x: vokzal(x))
train_coord['oao'] = train_coord.string.apply(lambda x: oao(x))
train_coord['health'] = train_coord.string.apply(lambda x: health(x))
train_coord['metro'] = train_coord.string.apply(lambda x: metro(x))
train_coord['child'] = train_coord.string.apply(lambda x: child(x))
train_coord['school'] = train_coord.string.apply(lambda x: school(x))
train_coord['giper'] = train_coord.string.apply(lambda x: giper(x))
train_coord['city'] = train_coord.string.apply(lambda x: city(x))
train_coord['road'] = train_coord.string.apply(lambda x: road(x))

test_coord['tc'] = test_coord.string.apply(lambda x: tc(x))
test_coord['air'] = test_coord.string.apply(lambda x: air(x))
test_coord['bc'] = test_coord.string.apply(lambda x: bc(x))
test_coord['vokzal'] = test_coord.string.apply(lambda x: vokzal(x))
test_coord['oao'] = test_coord.string.apply(lambda x: oao(x))
test_coord['health'] = test_coord.string.apply(lambda x: health(x))
test_coord['metro'] = test_coord.string.apply(lambda x: metro(x))
test_coord['child'] = test_coord.string.apply(lambda x: child(x))
test_coord['school'] = test_coord.string.apply(lambda x: school(x))
test_coord['giper'] = test_coord.string.apply(lambda x: giper(x))
test_coord['city'] = test_coord.string.apply(lambda x: city(x))
test_coord['road'] = test_coord.string.apply(lambda x: road(x))

In [38]:
train_coord.to_csv('train_coord.csv', index=False, encoding='utf-8')
test_coord.to_csv('test_coord.csv', index=False, encoding='utf-8')

In [39]:
data = pd.concat([train_coord, test_coord])

In [40]:
data

,action_lat,action_lon,string,tc,air,bc,vokzal,oao,health,metro,child,school,giper,city,road
0,59.844072,30.179153,о’кей 2 улица партизана германа лигово окр...,0,0,0,0,0,0,0,0,0,1,0,0
1,59.858198,30.229024,о’кей 31 к1 проспект маршала жукова юго-зап...,0,0,0,0,0,0,0,0,0,1,0,0
2,54.982358,82.892559,гранит 5 площадь карла маркса ленинский рай...,0,0,0,0,0,0,0,0,0,0,0,0
3,55.026997,82.920634,своя компания 17 красный проспект центральн...,0,0,0,0,0,0,0,0,0,0,0,0
4,54.964984,82.928976,48 улица ватутина кировский район новосибир...,0,0,0,0,0,0,0,0,0,0,0,0
5,55.025451,82.926714,travellers' coffee 1а каменская улица центр...,0,0,0,0,0,0,0,0,0,0,0,0
6,54.988900,82.862242,дневной стационар поликлиники 26 40 улица ко...,0,0,0,0,0,1,0,0,0,0,0,0
7,55.031843,82.919409,пирамида 29/1 красный проспект центральный ...,0,0,0,0,0,0,0,0,0,0,0,0
8,55.031051,82.919158,новосибирский облпотребсоюз 29 красный просп...,0,0,0,0,0,0,0,0,0,0,0,0
9,53.978282,79.243318,22в улица ленина городское поселение красноз...,0,0,0,0,0,0,0,0,0,0,1,0


In [228]:
lat = 55.639441
lon = 37.841934

nearby = np.zeros((250000, 9))
it = 0
for i in range(100):
    data['y'] = np.sqrt((data.action_lat - lat)**2 + (data.action_lon - lon)**2)
    tdata = data[data.y <= 0.03]
    if(tdata.shape[0] > 0):
        nearby[it] = np.sum(tdata.values[:,3:-1], axis=0)
        it += 1

In [221]:
np.sum(tdata.values[:,3:-1], axis=0)

array([7, 0, 0, 0, 0, 2, 15, 0, 4], dtype=object)

In [41]:
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

In [42]:
train_coord = pd.read_csv('train_coord.csv', encoding='utf-8')
test_coord = pd.read_csv('test_coord.csv',encoding='utf-8')
data = pd.concat([train_coord, test_coord])

for ep in [0.012]:
    ddd = {}
    ddd['products'] = [5411, 5499]
    ddd['meal'] = [5812, 5814]
    ddd['bank'] = [6011, 6010]
    ddd['apteka'] = [5912]
    ddd['hotel'] = [7011]
    ddd['parking'] = [7523]
    ddd['alhogol'] = [5921]
    ddd['car'] = [5541, 5533]
    ddd['univermag'] = [5311, 5999, 5331]
    ddd['entertainment'] = [7832, 7932, 7933]
    ddd['sport'] = [5941]
    ddd['hair'] = [7230]
    ddd['cosm'] = [5977]
    ddd['clothes'] = [5699, 5651, 5661, 5641]
    ddd['child_clo'] = [5641]
    ddd['zoomag'] = [5995]
    mcc_list = ['products', 'meal', 'bank', 'apteka', 'hotel', 'parking', 'alhogol', 'car', 'univermag',\
                'entertainment', 'sport', 'hair', 'cosm', 'clothes', 'child_clo', 'zoomag']

    os.chdir('/home/lenny/Documents/Python Scripts/Raiff/test_validation')
    uniq_cust = train.customer_id.unique()
    print(ep)
    clf = DBSCAN(eps = ep, p=2, min_samples=2)
    nearby = np.zeros((250000, 12))
    nearby2 = np.zeros((250000, 12))
    it = 0

    for u in tqdm_notebook(uniq_cust):
        temp = pd.read_csv('train_customers/' + u + '.csv')
        p = temp[['country','transaction_date','date_int','customer_id','pos_adress_lat', 'pos_adress_lon', 'terminal_id', 'amount', 'mcc', 'dow', 'hol', 'home_add_lat', 'home_add_lon']].rename(columns={'pos_adress_lat':'lat', 'pos_adress_lon':'lon'})
        a = temp[['country','transaction_date','date_int','customer_id','atm_address_lat', 'atm_address_lon', 'terminal_id','amount', 'mcc', 'dow', 'hol', 'home_add_lat', 'home_add_lon']].rename(columns={'atm_address_lat':'lat', 'atm_address_lon':'lon'})
        temp = pd.concat([p, a], axis=0)
        temp.dropna(inplace=True)
        temp = temp[(temp.country == 'RUS') | (temp.country == 'RU ')]


        if(temp.shape[0] > 0):
            temp['out'] = clf.fit_predict(temp[['lat', 'lon']].values)
            home = [max(temp['home_add_lat'].values), max(temp['home_add_lon'].values)]
#             home = [1]

            if(home[0] > 0):
                for cl in range(temp.out.max()+1):
                    temp2 = temp[temp.out == cl]
                    lat = temp2.lat.mean()
                    lon = temp2.lon.mean()
                    data['y'] = np.sqrt((data.action_lat - lat)**2 + (data.action_lon - lon)**2)
                    tdata = data[data.y <= 0.03]
                    if(tdata.shape[0] > 0):
                        nearby[it] = np.sum(tdata.values[:,3:-1], axis=0)
                        nearby2[it] = [int(i > 0) for i in np.sum(tdata.values[:,3:-1], axis=0)] 
                        it += 1
                    else:
                        it += 1
                    

0.012


A Jupyter Widget

In [43]:
a = np.concatenate((nearby[:it], nearby2[:it]),axis=1)

In [44]:
df = pd.DataFrame(a)

In [45]:
df.shape

(83021, 24)

In [46]:
df.to_csv('train_home_last_feat_2_12.csv', index=False)

In [31]:
pd.read_csv('train2_dbscan1_0.012_work.csv')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,customer_id
0,70.0,7.0,11.0,9.0,12.0,13.0,12.0,6.0,17.0,4.0,19.0,0.026746,0.039420,0.047637,6.0,59.845500,30.180000,3.565942,3.597486,0.0,32.0,38.0,3.669941,305.0,4.357143,212.0,86.675421,6.0,15.0,24.0,25.0,1.0,0.0,0.0,0.0,59.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,1.0,4.0,4.0,1.526293e-04,NaN,0.000806,NaN,NaN,0.003546,0dc0137d280a2a82d2dc89282450ff1b
1,70.0,7.0,11.0,9.0,12.0,13.0,12.0,6.0,17.0,4.0,19.0,0.026746,0.039420,0.047637,6.0,59.845500,30.180000,3.565942,3.597486,0.0,32.0,38.0,3.669941,305.0,4.357143,212.0,86.675421,6.0,15.0,24.0,25.0,1.0,0.0,0.0,0.0,59.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,2.0,2.0,3.0,5.181029e-02,NaN,0.052736,NaN,NaN,0.053192,0dc0137d280a2a82d2dc89282450ff1b
2,70.0,7.0,11.0,9.0,12.0,13.0,12.0,6.0,17.0,4.0,19.0,0.026746,0.039420,0.047637,6.0,59.845500,30.180000,3.565942,3.597486,0.0,32.0,38.0,3.669941,305.0,4.357143,212.0,86.675421,6.0,15.0,24.0,25.0,1.0,0.0,0.0,0.0,59.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,3.0,5.0,2.0,7.066577e-02,NaN,0.071602,NaN,NaN,0.072180,0dc0137d280a2a82d2dc89282450ff1b
3,70.0,7.0,11.0,9.0,12.0,13.0,12.0,6.0,17.0,4.0,19.0,0.026746,0.039420,0.047637,6.0,59.845500,30.180000,3.565942,3.597486,0.0,32.0,38.0,3.669941,305.0,4.357143,212.0,86.675421,6.0,15.0,24.0,25.0,1.0,0.0,0.0,0.0,59.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,4.0,1.0,1.0,8.998240e-02,NaN,0.090940,NaN,NaN,0.092022,0dc0137d280a2a82d2dc89282450ff1b
4,70.0,7.0,11.0,9.0,12.0,13.0,12.0,6.0,17.0,4.0,19.0,0.026746,0.039420,0.047637,6.0,59.845500,30.180000,3.565942,3.597486,0.0,32.0,38.0,3.669941,305.0,4.357143,212.0,86.675421,6.0,15.0,24.0,25.0,1.0,0.0,0.0,0.0,59.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,5.0,3.0,5.0,3.996114e-02,NaN,0.039503,NaN,NaN,0.041629,0dc0137d280a2a82d2dc89282450ff1b
5,70.0,7.0,11.0,9.0,12.0,13.0,12.0,6.0,17.0,4.0,19.0,0.026746,0.039420,0.047637,6.0,59.845500,30.180000,3.565942,3.597486,0.0,32.0,38.0,3.669941,305.0,4.357143,212.0,86.675421,6.0,15.0,24.0,25.0,1.0,0.0,0.0,0.0,59.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,6.0,6.0,6.0,2.961259e-01,NaN,0.296815,NaN,NaN,0.295625,0dc0137d280a2a82d2dc89282450ff1b
6,85.0,11.0,18.0,9.0,20.0,10.0,7.0,10.0,49.0,15.0,17.0,0.854931,4.001501,4.091811,14.0,55.777000,37.656169,3.340166,3.376171,0.0,49.0,36.0,3.749337,294.0,3.458824,179.0,62.443494,5.0,31.0,41.0,8.0,18.0,3.0,15.0,1.0,21.0,7.0,8.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.269281,2.0,0.0,95.459415,202.5,2.0,2.0,2.0,2.0,0.001290,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,2.0,1.0,1.0,7.548958e+00,2.685479e-01,0

In [254]:
data

,action_lat,action_lon,string,tc,air,bc,vokzal,oao,health,metro,child,school,y
0,59.844072,30.179153,о’кей 2 улица партизана германа лигово окр...,0,0,0,0,0,0,0,0,0,53.120964
1,59.858198,30.229024,о’кей 31 к1 проспект маршала жукова юго-зап...,0,0,0,0,0,0,0,0,0,53.072645
2,54.982358,82.892559,гранит 5 площадь карла маркса ленинский рай...,0,0,0,0,0,0,0,0,0,0.218126
3,55.026997,82.920634,своя компания 17 красный проспект центральн...,0,0,0,0,0,0,0,0,0,0.229469
4,54.964984,82.928976,48 улица ватутина кировский район новосибир...,0,0,0,0,0,0,0,0,0,0.178926
5,55.025451,82.926714,travellers' coffee 1а каменская улица центр...,0,0,0,0,0,0,0,0,0,0.224480
6,54.988900,82.862242,дневной стационар поликлиники 26 40 улица ко...,0,0,0,0,0,1,0,0,0,0.246435
7,55.031843,82.919409,пирамида 29/1 красный проспект центральный ...,0,0,0,0,0,0,0,0,0,0.234019
8,55.031051,82.919158,новосибирский облпотребсоюз 29 красный просп...,0,0,0,0,0,0,0,0,0,0.233555
9,53.978282,79.243318,22в улица ленина городское поселение красноз...,0,0,0,0,0,0,0,0,0,3.919176
